Compress from long to wide form

Caution: does not work properly. The Dissection into groups is meant to save memory with large files. However, I didn't check that the index changes when a group is created

In [1]:
import os
import pandas as pd
from tqdm import tqdm

main_dir = r'/home/janik.fechtelpeter/Documents/ordinal-bptt/results/lr_sl_tf_MRT1_EveryValidDay_Smoothed'
eval_dir = '00_summary_7stepsahead'


eval_dir = os.path.join(main_dir, eval_dir)
df = pd.read_csv(os.path.join(eval_dir, 'evaluation.csv'), index_col=0)

vary_by_step = ['ground_truth', 'prediction']
vary_by_step_and_feature = ['ground_truth', 'gt_mean', 'gt_var', 'train_mean', 'train_var', 'prediction']

constant_by_step_and_feature = df.columns.drop(labels=vary_by_step_and_feature + ['steps', 'feature']).to_list()

df_wide = []
grouper = df.groupby('model_id')
for id, group in tqdm(grouper, total=len(grouper)):

    group_wide = group.set_index(constant_by_step_and_feature + ['steps', 'feature'], append=True)
    group_wide = group_wide.unstack(['steps', 'feature'])
    group_wide = group_wide.groupby(['model_id', 'run', 'participant', 'train_on_data_until_timestep']).mean()
    df_wide.append(group_wide)

df_wide = pd.concat(df_wide, axis=0)
df_wide.to_csv(os.path.join(eval_dir, 'evaluation_compressed.csv'))


  0%|          | 0/18880 [00:17<?, ?it/s]


KeyError: 'train_on_data_until_timestep'

In [8]:
import os
print(os.path.getsize('/home/janik.fechtelpeter/Documents/ordinal-bptt/results/lr_sl_tf_MRT1_EveryValidDay_Smoothed/00_summary_7stepsahead/evaluation.csv') / (1024**3))
print(os.path.getsize('/home/janik.fechtelpeter/Documents/ordinal-bptt/results/lr_sl_tf_MRT1_EveryValidDay_Smoothed/00_summary_7stepsahead/evaluation_reduced.csv') / (1024**3))

54.76528852619231
0.8436242956668139


Decompress from wide to long form

In [ ]:
import os
import pandas as pd

main_dir = r'/home/janik.fechtelpeter/Documents/ordinal-bptt/results/lr_sl_tf_MRT1_EveryValidDay_Smoothed'
eval_dir = '00_summary_7stepsahead'


eval_dir = os.path.join(main_dir, eval_dir)
df_wide = pd.read_csv(os.path.join(eval_dir, 'evaluation_compressed.csv'))

df_wide = df_wide.stack(['steps', 'feature'])
df_wide.index = df.index.rename('wide_idx')

In [5]:
import sys
sys.path.append('..')
import eval_reallabor_utils

main_dir = r'/home/janik.fechtelpeter/Documents/ordinal-bptt/results/lr_sl_tf_MRT1_EveryValidDay_Smoothed'
eval_dir = '00_summary_7stepsahead'


eval_dir = os.path.join(main_dir, eval_dir)
df = pd.read_csv(os.path.join(eval_dir, 'evaluation.csv'), index_col=0)

df = eval_reallabor_utils.include_exclude_hypers(df, include_hyper={'tf_alpha':0.01, 'learning_rate':1e-3, 'lr_annealing':0, 'seq_len':18}, exclude_hyper={})

df.to_csv(os.path.join(eval_dir, 'evaluation_reduced.csv'))